## References

- https://rapids.ai/start.html
- https://github.com/rapidsai/cudf
- https://hub.docker.com/r/rapidsai/rapidsai/
- https://docs.rapids.ai/api/cudf/stable/10min.html
- https://github.com/beckernick/nersc-rapids-workshop
- https://towardsdatascience.com/heres-how-you-can-speedup-pandas-with-cudf-and-gpus-9ddc1716d5f2

### Videos

- [cuDF: RAPIDS GPU-Accelerated Dataframe Library" - Mark Harris (PyCon AU 2019)](https://www.youtube.com/watch?reload=9&v=lV7rtDW94do)
- [Introduction to cuDF - NERSC NVIDIA RAPIDS Workshop on April 14, 2020](https://www.youtube.com/watch?v=pXnEniQRAdQ)

Notes:

- It requires NVidia GPU

Prerequisites

    - NVIDIA Pascal™ GPU architecture or better
    - CUDA 10.1/10.2/11.0 with a compatible NVIDIA driver
    - Ubuntu 16.04/18.04 or CentOS 7
    - Docker CE v18+
    - nvidia-docker v2+
    
Installation

`
conda install -c nvidia -c rapidsai -c numba -c conda-forge -c defaults cudf
`

## Misc

check computer GPU
`
sudo lshw -numeric -C display
`

`
sudo lspci -v | less
`

Source:
https://www.howtogeek.com/508993/how-to-check-which-gpu-is-installed-on-linux/

Check Cuda Version
https://varhowto.com/check-cuda-version/
https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html

Cuda Guide:
https://developer.nvidia.com/cuda-downloads?target_os=Linux&target_arch=x86_64&target_distro=Ubuntu&target_version=1804&target_type=deblocal

Troubleshooting:
https://stackoverflow.com/a/64593288/2670476

## What is cuDF?

cuDF is a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating tabular data using a DataFrame style API.

In [1]:
input_file = 'data/data_1gb.csv'

In [2]:
import pandas as pd
import numpy as np
import cudf

pandas_df = pd.DataFrame({'a': np.random.randint(0, 100000000, size=100000000),
                          'b': np.random.randint(0, 100000000, size=100000000)})
                          
cudf_df = cudf.DataFrame.from_pandas(pandas_df)

In [3]:
pandas_df.head()

,a,b
0,15095380,25371605
1,31516221,77008762
2,34488765,51685408
3,27803421,9761639
4,93664344,52596724


In [4]:
cudf_df.head()

,a,b
0,15095380,25371605
1,31516221,77008762
2,34488765,51685408
3,27803421,9761639
4,93664344,52596724


In [5]:
%%time
pandas_df.a.mean()

CPU times: user 76.2 ms, sys: 24 µs, total: 76.2 ms
Wall time: 75.3 ms


49994356.54014489

In [6]:
%%time
cudf_df.a.mean()

CPU times: user 6.2 ms, sys: 4.31 ms, total: 10.5 ms
Wall time: 9.95 ms


49994356.54014489

In [7]:
%%time
pandas_df.merge(pandas_df, on='b')

CPU times: user 41 s, sys: 3.55 s, total: 44.6 s
Wall time: 44.6 s


,a_x,b,a_y
0,15095380,25371605,15095380
1,31516221,77008762,31516221
2,31516221,77008762,91299614
3,91299614,77008762,31516221
4,91299614,77008762,91299614
...,...,...,...
200033505,79609130,98817346,79609130
200033506,68413578,75914363,68413578
200033507,51414627,61111027,51414627
200033508,39177169,516351,39177169


In [8]:
%%time
cudf_df.merge(cudf_df, on='b')

MemoryError: std::bad_alloc: CUDA error at: /home/developer/miniconda3/include/rmm/mr/device/cuda_memory_resource.hpp:69: cudaErrorMemoryAllocation out of memory

In [10]:
import cudf
gdf = cudf.read_csv(input_file)
for column in gdf.columns:
    print(gdf[column].mean())

# gdf

MemoryError: std::bad_alloc: CUDA error at: /home/developer/miniconda3/include/rmm/mr/device/cuda_memory_resource.hpp:69: cudaErrorMemoryAllocation out of memory

In [20]:
import cudf, io, requests
from io import StringIO

url = "https://github.com/plotly/datasets/raw/master/tips.csv"
content = requests.get(url).content.decode('utf-8')

tips_df = cudf.read_csv(StringIO(content))
tips_df['tip_percentage'] = tips_df['tip'] / tips_df['total_bill'] * 100

# display average tip by dining party size
print(tips_df.groupby('size').tip_percentage.mean())

size
1    21.729202
2    16.571919
3    15.215685
4    14.594901
5    14.149549
6    15.622920
Name: tip_percentage, dtype: float64


## Dask-CUDA

In [16]:
# !conda install -c rapidsai dask-cudf

In [22]:
# source: https://github.com/rapidsai/cudf/issues/2288

import os
import gc
import timeit
import cudf as cu
import dask_cudf as dkcu
# x = cu.read_csv("data/data_1gb.csv", flush=True)
x = dkcu.read_csv("data/data_1gb.csv", flush=True)
x

,mno_ms_id,pos_time,mno_cell_id
npartitions=4,,,
,int64,object,int64
,...,...,...
,...,...,...
,...,...,...
,...,...,...


In [21]:
import time

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

ModuleNotFoundError: No module named 'dask_cuda'